# Import

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from datasets import load_dataset
import pandas as pd
import pickle
import json
import os

# Data

In [5]:
TRAIN_PATH = os.path.join('..', 'POS_Tagging', 'data', 'train')
TEST_PATH = os.path.join('..', 'POS_Tagging', 'data', 'test')
VOCAB_PATH = os.path.join('..', 'POS_Tagging', 'data', 'vocab', 'words.json')

In [6]:
dataset = load_dataset('parquet', data_files=[*[os.path.join(TRAIN_PATH, f'train_{i}.parquet') for i in range(9)],
                                    *[os.path.join(TEST_PATH, f'test_{i}.parquet') for i in range(9)]])

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

In [45]:
with open(VOCAB_PATH, 'r') as f:
    vocab = json.load(f)

In [48]:
vocab[0:10]

['თუკის',
 'ფირჩოსანნი',
 'დოკუმენტალისტიკიდან',
 'გაგემახვილებინა',
 'დააწებო',
 'ცილისწამებით',
 'თაყვანისვცე',
 'ითმინოო',
 'ანიტამ',
 'მოსამსახურეები']

# TF-IDF

In [4]:
vectorizer = TfidfVectorizer()

In [7]:
vectorizer.fit((" ".join(['<UNK>' if pos == 'Other' else '<NUM>' if pos == 'Num' else word for word, pos in zip(sent['mod_words'], sent['pos_tags']) if pos != 'Punct']) for sent in dataset['train']))

TfidfVectorizer()

In [24]:
with open(os.path.join('models', 'tfidf.pkl'), 'wb') as f:
    pickle.dump(vectorizer, f, protocol=pickle.DEFAULT_PROTOCOL)

In [7]:
with open(os.path.join('models', 'tfidf.pkl'), 'rb') as f:
    vectorizer = pickle.read(vectorizer, f, protocol=pickle.DEFAULT_PROTOCOL)

AttributeError: module 'pickle' has no attribute 'read'

In [2]:
feature_names = vectorizer.get_feature_names_out()
feature_names

NameError: name 'vectorizer' is not defined

In [154]:
df_idf = pd.DataFrame(vectorizer.idf_, index=feature_names, columns=["idf_weights"])

#inverse document frequency
potential_stop_words = list(df_idf.sort_values(by=['idf_weights']).head(300).index)

In [1]:
stop_words = [word for word in potential_stop_words if word not in 
                      set(['num', 'ქართველი', 'ქართული', 'ქვეყანაში', 'აშშ', 'განცხადებით', 
                           'ქვეყნის', 'ადამიანი', 'ადამიანის' ,'unk', 'განაცხადა',
                            'საქართველოს', 'წლის', 'შესახებ', 'საქართველოში', 
                            'თბილისში', 'ქართულ', 'ხელისუფლება', 'ხელისუფლების',  
                            'რუსეთის', 'საგარეო',  'ცნობილი', 'ცოტა', 'ძალიან', 'წელს', 
                            'თბილისის', 'თითქმის', 'ეროვნული', 'პარლამენტის', 'პოლიტიკური', 
                            'პრეზიდენტი', 'პრეზიდენტის',  'ხელი', 'დიდ', 'დიდი', 'აფხაზეთის',
                            'სამინისტროს', 'სამხედრო', 'სამხრეთ','საპარლამენტო', 'საქართველო',
                            'საქმე', 'საქმეთა', 'საჭირო', 'სახელმწიფო', 'აზრით', 'ალბათ', 'ამბობს',
                            'არჩევნების', 'აცხადებს', 'ახალ', 'ახალი', 'განმავლობაში', 'განსაკუთრებით',
                            'დავით', 'ინფორმაცია', 'ინფორმაციით', 'თავს', 'ხელს', 'ხელისუფლებას',
                            'ეკონომიკური', 'კაცი', 'მთავრობის', 'პრეზიდენტმა', 'ჩანს', 'შეიძლება', 'ერთად', 
                            'ერთ', 'ყველაზე', 'საერთაშორისო', 'მთელი', 'მინისტრის', 'თავი', 'გიორგი',
                            'მთავარი', 'დაიწყო', 'შეეხება', 'საკმაოდ', 'გუშინ', 'არაფერი', 'კარგი',  'მსოფლიო',
                            'ერთხელ', 'ხშირად', 'ყოველთვის',  'ბოლოს', 'მიხეილ',  'საკითხი',  'დღის',  'ადრე', 'არსებული',
                            'საარჩევნო', 'ნაწილი', 'მნიშვნელოვანი', 'ჯერჯერობით', 'პრემიერ',  'ყოფილი','ვიცი',
                            'საბჭოს', 'ნიშნავს',  'ხალხი', 'ევროპის', 'მინისტრი', 'მსოფლიოს', 'მხრიდან',
                            'მინდა', 'მეტად',])]
len(stop_words), stop_words[:120] 

NameError: name 'potential_stop_words' is not defined

In [101]:

import nltk
from nltk.corpus import stopwords
 
nltk.download('stopwords')
print(len(stopwords.words('english')))

179


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [39]:
print(vectorizer.transform(['მე შენ და ის ']).todense())

[[0. 0. 0. ... 0. 0. 0.]]
